# automation of warehouse reporting pipeline

Note from author: the code comments and markdown notes are written in russian, as the script was used in a local environment, to make it clear what is happening for people who are not fluent in english.

# 1. Loading libraries and tabels

In [1]:
# Установить при отсутствии
%pip install xlrd
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Импортируем используемые библиотеки
import os
import pandas as pd
from datetime import datetime

In [3]:
# Автоматически определяем путь к input_docs
current_dir = os.getcwd()  # директория, где работаем
input_path = os.path.join(current_dir, 'input_docs')

In [4]:
# Загружаем таблицы в переменную
paths = {}

for tab in ['control.xls', 'storage1_main.xlsx', 'storage2_ip.xlsx']:
    file_path = os.path.join(input_path, tab)
    paths[tab] = file_path

control = pd.read_excel(paths.get('control.xls'))
storage1_main = pd.read_excel(paths.get('storage1_main.xlsx'))
storage2_ip = pd.read_excel(paths.get('storage2_ip.xlsx'))

# 2. Introduction and pre-processing

In [5]:
# Создаем пару название датафрейма - датафрейм для удобства вывода данных
dfs = {
    'control' : control,
    'storage1_main' : storage1_main,
    'storage2_ip' : storage2_ip
}

# Выводим первые 5 строк каждого датафрема
for name, df in dfs.items():
    print(name)
    display(df.head())
    print()
    df.info()
    print('-------------------------------------------------------------------------------------------------------------')

control


,item_name,quantity
0,Авт. выключатель 2п С 6А,2.0
1,Адаптер RS-232 для ВКТ,1.0
2,Адаптер USB-LIN для РС,1.0
3,Адаптер АПС-70 (СОМ-порт компьютера - оптопорт...,2.0
4,Адаптер АПС-71 (USB-порт компьютера - оптопорт...,2.0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   item_name  252 non-null    object 
 1   quantity   241 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.1+ KB
-------------------------------------------------------------------------------------------------------------
storage1_main


,item_name,quantity
0,Авт. выключатель 2п С 6А,20.0
1,Адаптер RS-232 для ВКТ,1.0
2,Адаптер USB-LIN для РС,1.0
3,Адаптер АПС-70 (СОМ-порт компьютера - оптопорт...,2.0
4,Адаптер АПС-71 (USB-порт компьютера - оптопорт...,2.0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   item_name  226 non-null    object 
 1   quantity   212 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.7+ KB
-------------------------------------------------------------------------------------------------------------
storage2_ip


,item_name,quantity
0,Адаптер АПС-70 (СОМ-порт компьютера - оптопорт...,1
1,Адаптер АПС-77 (COM пк -RS232),3
2,"Адаптер записи коэффицентов ""Комбик-Т"" (РМД)",1
3,"Адаптер записи коэффицентов ""Компакт Т-21"" (РМД)",3
4,Антенна mini GSM SMA (5db) не основная,1



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_name  63 non-null     object
 1   quantity   63 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
-------------------------------------------------------------------------------------------------------------


В некоторых таблицах присутствуют пропуски: `control` и `storage1_main`. В данном случае мы знаем, что данные позиции отсутствуют на складе. Пропуски классифицируем как **MAR** - объясняются наблюдаемыми данными. Выведем строки с пропусками, чтобы ознакомится, и заполним нулями.

In [6]:
def show_nulls(df):
# Функция принимает датафрейм на вход и выводит данные по пропускам 
# в столбцах: количество и их процент от общего количества строк
    
    result = pd.DataFrame({
        'Количество пропусков' : df.isna().sum(),
        'Процент пропусков (%)' : round(df.isna().mean() * 100, 2),
        }).sort_values(by='Процент пропусков (%)', ascending=False)

    return display(result)

In [7]:
for name, df in dfs.items():
    if name != 'storage2_ip':
        print(f'Анализ пропусков в {name}:')
        show_nulls(df)
        display(df[df.isna().any(axis=1)].head(5))

Анализ пропусков в control:


,Количество пропусков,Процент пропусков (%)
quantity,11,4.37
item_name,0,0.00


,item_name,quantity
137,Монтажный комплект Ду-32,NaN
146,"МФ(5.2.2)-Ду 20 класс Б, Уст. 0,001 м3/имп.",NaN
148,"МФ(5.2.2)-Ду 32 класс Б, Уст. 0,001 м3/имп",NaN
149,"МФ(5.2.2)-Ду 50 класс Б, Уст. 0,01 м3/имп.",NaN
156,Питерфлоу РС Ду 50-72 кл. А - сенд. расходомер...,NaN


Анализ пропусков в storage1_main:


,Количество пропусков,Процент пропусков (%)
quantity,14,6.19
item_name,0,0.00


,item_name,quantity
116,Монтажный комплект Ду-32,NaN
122,"МФ(5.2.2)-Ду 20 класс Б, Уст. 0,001 м3/имп.",NaN
123,"МФ(5.2.2)-Ду 25 класс Б, Уст. 0,001 м3/имп",NaN
124,"МФ(5.2.2)-Ду 32 класс Б, Уст. 0,001 м3/имп",NaN
125,"МФ(5.2.2)-Ду 50 класс Б, Уст. 0,01 м3/имп.",NaN


In [8]:
# Заполним пропуски нулями
for name, df in dfs.items():
    df['quantity'] = df['quantity'].fillna(0)

    
    # Заодно проверим на дубликаты
    print(f"Количество дубликатов товаров в {name}: {df['item_name'].duplicated().sum()}")

Количество дубликатов товаров в control: 0
Количество дубликатов товаров в storage1_main: 0
Количество дубликатов товаров в storage2_ip: 0


In [9]:
# Переименуем столбцы, чтобы после присоединения идентифицировать столбцы
for name, df in dfs.items():
    dfs[name] = df.rename(columns={'quantity' : f'{name}_quantity'})

control = dfs.get('control')
storage1_main = dfs.get('storage1_main')
storage2_ip = dfs.get('storage2_ip')

# 3. Connecting tables and building the report

Главная таблица у нас `storage1_main`, количество товара здесь строго отражает фактический остаток. Нам нужно найти различия товара между количеством в этой таблице и остальными.

In [10]:
# Соединяем таблицы, пользуемся full join - в данном случае пропуски несут ключевую информацию об отсутствии товара, их так же заменим на нули
 
merged_pre = pd.merge(storage1_main,
                      control,
                      how='outer',
                      on='item_name')

merged = pd.merge(merged_pre,
                  storage2_ip,
                  how='outer',
                  on='item_name')

In [11]:
# Заполняем пропуски
merged = merged.fillna(0)

In [12]:
def shuffle(row):
    # В функцию передается строка датафрейма
    # Задача скорректировать количество товара на storage1_main_quantity с учтом storage2_ip_quantity, а именно чтобы разница между main и control
    # нивелировалась с помощью ip
    # Функция работает по следующей логике:
    # если кол-во товара storage1_main_quantity < control_quantity, но при этом storage2_ip_quantity > 0
    # тогда на склад storage1_main_quantity перемещаем столько товара, чтобы control_quantity = storage1_main_quantity
    # иначе возвращаем исходное значение
    upd_row = row.copy()
    if upd_row['storage1_main_quantity'] < upd_row['control_quantity'] and upd_row['storage2_ip_quantity'] > 0:
        diff = upd_row['control_quantity'] - upd_row['storage1_main_quantity']
        if upd_row['storage2_ip_quantity'] <= diff:
            result = upd_row['storage1_main_quantity'] + upd_row['storage2_ip_quantity']
            return result
        elif upd_row['storage2_ip_quantity'] > diff:
            result = upd_row['storage2_ip_quantity'] + diff
            return result
    else: 
        return upd_row['storage1_main_quantity']

In [13]:
merged['corrected_main_quantity'] = merged.apply(shuffle, axis=1)

In [14]:
# Разница контроля и основного склада
merged['control_except_main'] = merged['control_quantity'] - merged['corrected_main_quantity']

In [15]:
# Положительные расхождения с control
positive_result = merged[merged['control_except_main'] > 0]

In [16]:
# Негативные расхождения с control
negative_result = merged[merged['control_except_main'] < 0]

In [17]:
# Для контроля происходящего раскомментить, строка для образца
# merged.loc[[247]]

# 4. Report presentation

Собираем два отчета для "положительных" и "отрицательных" товаров

In [18]:
positive_result_final = positive_result[['item_name', 'control_except_main']]

positive_result_final = (
    positive_result_final
        .rename(columns={'control_except_main' : 'positive_result'})
        .sort_values('item_name', ascending=True)
)

In [19]:
negative_result_final = negative_result[['item_name', 'control_except_main']]

negative_result_final = (
    negative_result_final
        .rename(columns={'control_except_main' : 'negative_result'})
        .sort_values('item_name', ascending=True)
)

In [20]:
def save_reports(report1_df=positive_result_final, 
                 report2_df=negative_result_final, 
                 current_dir=current_dir):
    #Функция сохраняет датафреймы в папку output_docs
    
    # Путь к папке output_docs (относительно скрипта)
    output_dir = os.path.join(current_dir, "output_docs")
    
    # Создаем папку если ее нет
    os.makedirs(output_dir, exist_ok=True)
    
    # Генерируем имена файлов с временной меткой
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Пути к файлам
    file1_path = os.path.join(output_dir, f"1_positive_result_{timestamp}.xlsx")
    file2_path = os.path.join(output_dir, f"2_negative_result_{timestamp}.xlsx")
    
    # Сохраняем в Excel
    report1_df.to_excel(file1_path, index=False)
    report2_df.to_excel(file2_path, index=False)
    
    print(f"✓ Отчеты сохранены в папку: {output_dir}")
    print(f"  Report 1: {os.path.basename(file1_path)}")
    print(f"  Report 2: {os.path.basename(file2_path)}")
    
    return file1_path, file2_path

In [21]:
save_reports()

✓ Отчеты сохранены в папку: E:\2_CODE_WORKS\Jupyter Notebook files\Pet projects\automation of warehouse reporting\output_docs
  Report 1: 1_positive_result_20260209_221325.xlsx
  Report 2: 2_negative_result_20260209_221325.xlsx


('E:\\2_CODE_WORKS\\Jupyter Notebook files\\Pet projects\\automation of warehouse reporting\\output_docs\\1_positive_result_20260209_221325.xlsx',
 'E:\\2_CODE_WORKS\\Jupyter Notebook files\\Pet projects\\automation of warehouse reporting\\output_docs\\2_negative_result_20260209_221325.xlsx')